# 03. Modeling – 시계열/수요 예측
## 3-1. 데이터 로드 & train/test 분리
## 3-2. 베이스라인 모형
## 3-3. ML 모델 1: 선형회귀
## 3-4. ML 모델 2: 랜덤포레스트 (또는 XGBoost 등)(머신러닝 예시)
## 3-5. 성능 비교 (RMSE, MAE, R² 테이블로 정리)

In [ ]:
import pandas as pd
import numpy as np

from src.data_loader import load_csv
from src.modeling_timeseries import (
    train_test_split_time,
    naive_baseline_predict,
    train_linear_regression,
    train_random_forest,
    evaluate_regression,
)

# 1. 데이터 로드
df = load_csv("data/processed/bike_features.csv")  # 각자 파일명 수정
target_col = "demand"

feature_cols = [
    "year", "month", "dayofweek", "hour",
    "demand_lag1", "demand_lag24",
    "demand_roll3", "demand_roll7", "demand_roll24",
]

# NaN 제거 (혹시 남아있다면)
df = df.dropna(subset=feature_cols + [target_col]).reset_index(drop=True)

In [ ]:
# train/test 분리 (시간 순서 유지)
X_train, X_test, y_train, y_test = train_test_split_time(
    df, feature_cols, target_col, test_size=0.2
)

len(X_train), len(X_test)

In [ ]:
# 2. 베이스라인 – 마지막 값 그대로
y_pred_naive = naive_baseline_predict(y_train, horizon=len(y_test))
metrics_naive = evaluate_regression(y_test, y_pred_naive)
metrics_naive

In [ ]:
# 3. 선형회귀
lr_model = train_linear_regression(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
metrics_lr = evaluate_regression(y_test, y_pred_lr)
metrics_lr

In [ ]:
# 4. 랜덤포레스트(예시)
rf_model = train_random_forest(X_train, y_train, n_estimators=300, max_depth=None)
y_pred_rf = rf_model.predict(X_test)
metrics_rf = evaluate_regression(y_test, y_pred_rf)
metrics_rf

In [ ]:
# 5. 성능 비교 테이블
results = pd.DataFrame([
    {"model": "naive", **metrics_naive},
    {"model": "linear_regression", **metrics_lr},
    {"model": "random_forest", **metrics_rf},
])

results
markdown
코드 복사


# 모델링 요약

- 베이스라인 대비 개선 정도:
  - naive vs LR vs RF 성능 비교 (RMSE/MAE/R²)

- 인사이트:
  - (예) 시계열 피처/날씨 피처 포함 시 오차 감소
  - (예) 과적합 여부, 복잡한 모델이 정말 필요한지 등